## HW03 Notebook

Complete the following notebook, as described in the PDF for Homework 03 (included in the download with the starter code). Submit the following:
1. This notebook file and `hw3.py`, along with your COLLABORATORS.txt file, to the Gradescope link for code.
2. A PDF of this notebook and all of its output, once it is completed, to the Gradescope link for the PDF.

__NOTE__: The purpose of this notebook is to demonstrate the functionality implemented in `hw3.py`. As part of this demo, all analysis (i.e., questions that prompt for a short answer) are to be added to the notebook. Keep the order of the problems as listed in the assignment description. Furthermore, cells are provided as placeholders for each response; however, cells can be added as needed.

Please report any questions to the [class Piazza page](https://piazza.com/class/lcwv1h9p2a11ai).

### Import required libraries.

In [1]:
import os
import numpy as np
import pandas as pd

import warnings

import sklearn.linear_model
import sklearn.metrics
from hw3 import calc_confusion_matrix_for_threshold
from hw3 import calc_percent_cancer
from hw3 import calc_binary_metrics
from hw3 import predict_0_always_classifier
from hw3 import calc_accuracy
from hw3 import print_perf_metrics_for_threshold
from hw3 import calc_perf_metrics_for_threshold

from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline
plt.style.use('seaborn-v0_8') # pretty matplotlib plots

#### 1) Function to calculate TP, TN, FP, and FN.
The following four calls to the function `calc_binary_metrics` to test it. This way, the function can be tested for several edge cases. ***Don't modify this.***

In [2]:
all0 = np.zeros(10)
all1 = np.ones(10)
TP, TN, FP, FN = calc_binary_metrics(all0, all1)
print(f"0 vs 1\n========\nTP: {TP}\nTN: {TN}\nFP: {FP}\nFN: {FN}")

0 vs 1
TP: 0.0
TN: 0.0
FP: 10.0
FN: 0.0


In [3]:
TP, TN, FP, FN = calc_binary_metrics(all1, all0)
print(f"1 vs 0\n========\nTP: {TP}\nTN: {TN}\nFP: {FP}\nFN: {FN}")

1 vs 0
TP: 0.0
TN: 0.0
FP: 0.0
FN: 10.0


In [4]:
TP, TN, FP, FN = calc_binary_metrics(all1, all1)
print(f"1 vs 1\n========\nTP: {TP}\nTN: {TN}\nFP: {FP}\nFN: {FN}")

1 vs 1
TP: 10.0
TN: 0.0
FP: 0.0
FN: 0.0


In [5]:
TP, TN, FP, FN = calc_binary_metrics(all0, all0)
print(f"0 vs 0\n========\nTP: {TP}\nTN: {TN}\nFP: {FP}\nFN: {FN}")

0 vs 0
TP: 0.0
TN: 10.0
FP: 0.0
FN: 0.0


### Load the dataset.

The following should ***not*** be modified.

After it runs, the various arrays it creates will contain the 2- or 3-feature input datasets.

In [6]:
# Load the x-data and y-class arrays
x_train = np.loadtxt('./data/x_train.csv', delimiter=',', skiprows=1)
x_test = np.loadtxt('./data/x_test.csv', delimiter=',', skiprows=1)

y_train = np.loadtxt('./data/y_train.csv', delimiter=',', skiprows=1)
y_test = np.loadtxt('./data/y_test.csv', delimiter=',', skiprows=1)

Inspect Data. The following should ***not*** be modified.

In [7]:
feat_names = np.loadtxt(f'data/x_train.csv', delimiter=',', dtype=str, max_rows=1)
print(f"features: {feat_names}\n")
target_name = np.loadtxt(f'data/x_test.csv', delimiter=',', dtype=str, max_rows=1)
df_sampled_data = pd.DataFrame(x_test, columns=feat_names)
df_sampled_data[str(target_name)] = y_test
df_sampled_data.sample(15)

features: ['age' 'famhistory' 'marker']



,age,famhistory,marker,['age' 'famhistory' 'marker']
169,64.29861,0.0,2.843979,0.0
49,78.46741,0.0,0.267086,0.0
94,57.14237,1.0,1.384837,0.0
122,66.15447,0.0,0.987860,0.0
72,60.23331,0.0,0.720714,0.0
62,64.22443,0.0,2.231875,0.0
16,62.85834,1.0,1.623482,0.0
129,59.16642,1.0,0.166644,0.0
9,68.42139,0.0,0.959714,0.0
130,62.93052,0.0,0.077108,0.0


### 2) Compute the fraction of patients with cancer.

Complete the following code.  Your solution needs to ***compute*** these values from the training and testing sets (i.e., don't simply hand-count and print the values).

In [8]:
#TODO: modify these prints
tr_percent = calc_percent_cancer(y_train)
te_percent = calc_percent_cancer(y_test)

print("Percent of data that has_cancer on TRAIN: %.3f" % tr_percent)
print("Percent of data that has_cancer on TEST : %.3f" % te_percent)

Percent of data that has_cancer on TRAIN: 14.035
Percent of data that has_cancer on TEST : 13.889


### 3) The predict-0-always baseline

#### (i) Compute the accuracy of the always-0 classifier.

Complete the functions to compute and calculate the accuracy of the always-0 classifier on validation and test outputs.

In [9]:
#TODO: implement predict_0_always_classifer()
y_train_pred = predict_0_always_classifier(x_train)
y_test_pred = predict_0_always_classifier(x_test)

acc_train = calc_accuracy(*calc_binary_metrics(y_train, y_train_pred))
acc_test = calc_accuracy(*calc_binary_metrics(y_test, y_test_pred))
print("acc on TRAIN: %.3f" % acc_train) #TODO: modify these values
print("acc on TEST : %.3f" % acc_test)

acc on TRAIN: 0.860
acc on TEST : 0.861


#### (ii) Print a confusion matrix for the always-0 classifier.

Add code below to generate a confusion matrix for the always-0 classifier on the test set.

In [10]:
# TODO: call print(calc_confusion_matrix_for_threshold(...))
print(calc_confusion_matrix_for_threshold(y_true_N=y_test, y_proba1_N=y_test_pred)) # args : y_true_N, y_proba1_N, thresh=0.5

Predicted    0  1
True             
0          155  0
1           25  0


#### (iii) Reflect on the accuracy of the always-0 classifier.

**Answer**: TODO
(2) You will see reasonable accuracy for the simple baseline classifier. Why not just use it for this task? Your answer, written into the notebook as
text, should detail the pluses and minuses of using this simple classifier

This is pretty good accuracy at roughly 86%. Since we are always assuming there is no cancer; however, that is very dangerous if 14% of people have cancer and we choose not to biopsy them. That is unacceptable. If this was a social media algorithm like a recommender, 86% is plenty good, but life and death needs far better accuracy. It is safer to biopsy all patients to catch the 14% but you don't want to biopsy an unnecessary 86% on the other hand. That is why we can't use only a always-0 or always-1 predictor since surgery and people's lives are the consequences.

DO THISSSSSSSSSSSSSSSS

#### (iv) Analyze the various costs of using the always-0 classifier.

**Answer**: TODO

Additional costs of not taking biopsies of 14% of patients that have cancer beyond death is the doctors and/or hospital can be sued for negligence or medical malpractice.

DO THISSSSSSSSSSSSSSSS

### 4: Basic Perceptron Models

#### (i) Normalize data

In [11]:
from hw3 import standardize_data

#TODO
X_train, X_test = standardize_data(X_train=x_train, X_test=x_test)

# print(X_train)

ValueError: Expected 2D array, got 1D array instead:
array=[63.78864 60.21136 70.18679 64.85663 68.6578  71.34671 74.77879 60.76143
 62.87925 69.83581 58.65945 65.73421 72.44105 62.32544 64.59198 58.01468
 61.01774 62.45436 65.7565  65.81664 63.54927 67.13424 63.36813 67.97187
 67.18363 64.24229 63.94371 69.81638 65.50663 62.34288 64.58864 54.34343
 76.31532 70.89838 66.54893 67.78315 76.53217 59.40677 74.09642 58.00622
 65.99633 68.49203 59.8314  67.96156 70.39814 67.99323 66.97784 65.22306
 64.05421 75.53773 58.62805 68.316   70.09425 61.86872 63.57636 57.40099
 69.75618 66.46    63.07719 61.36214 54.02065 71.36636 67.21446 59.90642
 72.66306 66.35931 67.05412 61.98321 62.9514  64.32439 61.20963 70.28268
 63.09303 66.16218 54.60774 65.87422 59.57447 65.18739 70.12766 63.00902
 63.69997 68.29043 60.44851 63.37096 68.61874 63.60289 62.10342 68.16631
 63.92532 67.7188  66.06847 61.40981 61.69149 64.83163 60.76701 69.73871
 65.49047 64.94543 64.36356 59.46868 54.69748 66.83841 68.26522 7

#### (ii) Create a basic `Perceptron` classifier

Fit a perceptron to the training data.  Print out accuracy on this data, as well as on testing data.  Print out a confusion matrix on the testing data.

In [ ]:
#TODO: train a basic perceptron model using default parameter values, and modify these accuracies below

print("acc on TRAIN: %.3f" % 0)
print("acc on TEST : %.3f" % 0)

print("")
print("Confusion matrix for TEST:")
# TODO: call print(calc_confusion_matrix_for_threshold(...))

# print(calc_confusion_matrix_for_threshold(...))

#### (iii) Compare the `Perceptron` to the always-0 classifier.

**Answer**:  TODO


#### (iv) Generate a series of regularized perceptron models
Each model will use a different `alpha` value, multiplying that by the L2 penalty.  You will record and plot the accuracy of each model on both training and test data.

In [ ]:
train_accuracy_list = list()
test_accuracy_list = list()

# TODO: create, fit models here and record accuracy of each (Implement functions needed)

##### Plot accuracy on train/test data across the different alpha values plotted on a logarithmic scale. Make sure to show title, legends, and axis labels.

In [ ]:
# TODO make plot
plt.xlabel('log_10(alpha)');
plt.ylabel('Accuracy');

# TODO add legend, titles, etc. set x-scale appropriately
# plt.legend(...);

#### (iv) Discuss what the plot is showing you.

**Answer**: TODO

### 5: Decision functions and probabilistic predictions

#### (a) Create two new sets of predictions

Fit `Perceptron` and `CalibratedClassifierCV` models to the data.  Use their predictions to generate ROC curves.

In [ ]:
# TODO: fit a Perceptron and generate its decision_function() over the test data.

# TODO: Build a CalibratedClassifierCV, using a Perceptron as its base_estimator,
#       and generate its probabilistic predictions over the test data.

In [ ]:
# TODO something like: fpr, tpr, thr = sklearn.metrics.roc_curve(...)

plt.ylim([0, 1]);
plt.legend(loc='lower right')
plt.xlabel("False Postive rate (FPR)");
plt.ylabel("True Postive rate (TPR)");

In [ ]:
print("AUC on TEST for Perceptron: %.3f" % 0) #TODO: modify these values
print("AUC on TEST for probabilistic model: %.3f" % 0)

#### (b) Discuss the results above

**Answer**: TODO

#### (c) Compute model metrics for different probabilistic thresholds

Complete `calc_perf_metrics_for_threshold` that takes in a set of correct outputs, a matching set of probabilities generated by a classifier, and a threshold at which to set the positive decision probability, and returns a set of metrics if we use that threshold.

#### (d) Compare the probabilistic classifier across multiple decision thresholds

Try a range of thresholds for classifying data into the positive class (1).  For each threshold, compute the true positive rate (TPR) and positive predictive value (PPV).  Record the best value of each metric, along with the threshold that achieves it, and the *other* metric at that threshold.

In [ ]:
# TODO: test different thresholds to compute these values
best_TPR = 0
best_PPV_for_best_TPR = 0
best_TPR_threshold = 0

best_PPV = 0
best_TPR_for_best_PPV = 0
best_PPV_threshold = 0

In [ ]:
print("Best TPR threshold: %.4f => TPR: %.4f; PPV: %.4f" % (best_TPR_threshold, best_TPR, best_PPV_for_best_TPR))
print("Best PPV threshold: %.4f => PPV: %.4f; TPR: %.4f" % (best_PPV_threshold, best_PPV, best_TPR_for_best_PPV))

#### (e) Exploring different thresholds

#### (i) Using default 0.5 threshold.

Generate confusion matrix and metrics for probabilistic classifier, using threshold 0.5.

In [ ]:
best_thr = 0.5
print("ON THE TEST SET:")
print("Chosen best threshold = %.4f" % best_thr)
print("")
# TODO: print(calc_confusion_matrix_for_threshold(...))
print("")
# TODO: print_perf_metrics_for_threshold(...)

#### (ii) Using threshold with highest TPR.

Generate confusion matrix and metrics for probabilistic classifier, using threshold that maximizes TPR.

In [ ]:
best_thr = best_TPR_threshold
print("ON THE TEST SET:")
print("Chosen best threshold = %.4f" % best_thr)
print("")
# TODO: print(calc_confusion_matrix_for_threshold(...))
print("")
# TODO: print_perf_metrics_for_threshold(...)

#### (iii) Using threshold with highest PPV.

Generate confusion matrix and metrics for probabilistic classifier, using threshold that maximizes PPV.

In [ ]:
best_thr = best_PPV_threshold
print("ON THE TEST SET:")
print("Chosen best threshold = %.4f" % best_thr)
print("")
# TODO: print(calc_confusion_matrix_for_threshold(...))
print("")
# TODO: print_perf_metrics_for_threshold(...)

#### (iv) Compare the confusion matrices from (a)–(c) to analyze the different thresholds.

**Answer**: TODO